# Biased and fair coin sequences of arbitrary length

Contents:
* [Installing the parallel_simulations module](#installing-the-parallel_simulations-module)
* [Importing the PipelineOptions class](#import_options) 
* [Importing the `ParallelMCBattery` class](#import_class)
* [Battery configurations, and using the simulations instance](#battery_config)
* [Defining the statistical model](#stats_model)
* [Creating the simulations configurations](#simulations_configs)
* [Simulating with minimal configurations](#simulations_min_config)
* [Sample inspection](#sample_insp)
* [Simulating with output file paths specified via `output_paths`](#simulate_output)
* [Simulating by completing a pre-existing sequence with `starting_point`](#simulate_start)

<a id="install"></a>
### Installing the parallel_simulations module

In [ ]:
%pip install -q -e "git+https://github.com/vladimirrotariu/parallel-monte-carlo-simulations#egg=parallel_simulations&subdirectory=src"

<a id="import_options"></a>
### Importing the PipelineOptions class

In [2]:
import sys
from apache_beam.options.pipeline_options import PipelineOptions

<a id="import_class"></a>
### Importing the ParallelMCBattery class

In [3]:
from parallel_simulations import ParallelMCBattery

<a id="battery_config"></a>
### Battery configurations, and using the simulations instance

One may configure the Monte Carlo parallel battery by choosing the desired random number generator, in this case [Philox](https://numpy.org/doc/stable/reference/random/bit_generators/philox.html#philox-counter-based-rng), and the pipeline options instance of the class [PipelineOptions](https://beam.apache.org/releases/pydoc/2.33.0/apache_beam.options.pipeline_options.html#apache_beam.options.pipeline_options.PipelineOptions) of Apache Beam, for this example choosing for simplicity the default settings, which means the pipeline runs on the local `Direct Runner`.

In [4]:
# as we provide the pipeline_options programatically, and not via CLI
sys.argv = [""]
#####################################################################

options = PipelineOptions()

battery_configs = {"rng" : "Philox", "pipeline_options": options}

battery_parallel_MC = ParallelMCBattery(battery_configs=battery_configs)

<a id="stats_model"></a>
### Defining the statistical model

We are interested to model two distinct **parallelizable** sequences of heads 'H' and tails 'T' generated by simulating tossing coins of a given `bias`, which is a list of `parameters`, corresponding in this case to a unique parameter.

In [5]:
def CoinSequence(number_flips, rng, bias):
    return ["H" if rng.random() <= bias[0] else "T" for _ in range(number_flips)]

models = [CoinSequence, CoinSequence]

<a id="simulations_configs"></a>
### Creating the simulations configurations

To configure the simulations for these models, one further uses a list of dictionaries, each dictionary corresponding to one of the `models`.

In [6]:
# 100,000 sequences, each of 16 'H' or 'T' generated by simulating tossing a fair coin
fair_coin_config = {"parameters": [0.5], "number_simulations" : 100000, "number_points": 16}

# 60,000 sequences, each of 32 'H' or 'T' generated by simulating tossing a biased coin
biased_coin_config = {"parameters": [0.7], "number_simulations" : 60000, "number_points": 32}

simulation_configs = [fair_coin_config, biased_coin_config]

<a id="simulations_min_configs"></a>
### Simulating with minimal configurations

And now one may perform the Monte Carlo simulations configured above, having in mind that without specifying `output_paths`, the output csv files will be written by default in the directory where it is executed the Python script which calls the object method `simulate`:

In [7]:
battery_parallel_MC.simulate(models, simulation_configs)

<a id="sample_insp"></a>
### Sample inspection

As we did not specify the `output_paths`, the output files names' are created with the index corresponding from the `models` list

In [8]:
!head -n 12 0.txt

H,H,H,H,T,H,T,H,H,H,T,H,T,T,H,H
T,H,H,H,H,H,H,T,T,T,H,H,H,T,T,H
T,H,T,T,T,H,H,T,T,H,H,T,H,H,H,T
T,H,T,H,T,T,T,T,H,H,H,H,H,T,T,T
T,H,T,H,H,H,H,T,H,H,H,T,H,T,H,T
H,T,T,T,T,T,H,H,T,H,H,H,T,H,H,T
T,T,T,T,T,T,H,H,T,T,H,T,H,H,H,H
H,H,T,T,H,H,H,T,H,H,T,H,H,T,H,H
H,H,H,H,T,H,H,T,H,H,T,H,H,H,H,H
T,T,T,H,T,H,T,H,T,T,T,H,T,H,H,T
H,H,H,T,H,H,T,H,T,H,T,T,T,T,T,T
T,H,H,H,T,H,H,H,T,T,T,H,T,T,T,T


In [9]:
!head -n 12 1.txt

H,H,H,T,H,H,H,H,H,T,T,H,H,H,H,H,H,T,H,H,H,H,H,H,H,H,H,H,T,H,H,T
H,T,H,T,T,H,H,H,H,H,T,H,T,H,T,T,T,T,H,H,H,H,H,H,T,H,H,H,T,H,H,H
H,H,H,H,T,H,H,H,T,T,T,H,H,H,H,H,H,H,H,H,T,T,T,T,T,H,T,T,T,H,H,H
H,T,T,H,H,T,H,T,T,H,T,T,T,H,H,T,H,T,H,H,T,H,T,T,T,T,T,H,H,H,H,H
H,H,H,H,H,T,H,H,H,H,H,H,T,T,T,H,H,H,H,H,T,H,H,H,H,H,H,H,T,T,H,H
H,H,H,H,H,H,H,H,H,H,H,T,H,H,T,T,H,H,T,H,H,H,H,H,T,H,T,H,H,T,H,H
H,H,H,H,T,T,H,T,H,T,H,H,H,H,T,T,T,H,H,H,T,T,T,T,H,T,H,H,H,T,H,H
T,H,T,H,T,H,T,T,T,T,T,T,H,H,H,H,H,T,T,H,H,T,H,T,H,H,H,T,T,T,H,T
T,T,T,H,H,T,H,T,H,H,H,H,T,T,H,H,H,H,H,T,T,H,H,H,H,H,H,H,T,H,H,T
H,H,T,H,T,H,H,T,T,T,H,H,T,H,T,H,T,H,H,H,H,T,H,H,H,T,H,H,H,H,H,H
H,H,T,H,H,T,H,H,H,H,H,H,H,T,T,H,H,H,H,H,H,H,T,H,H,H,H,H,H,H,H,T
H,H,T,T,H,H,T,T,H,H,H,H,H,H,T,T,H,T,H,T,H,T,H,H,T,H,H,T,H,T,H,H


<a id="simulate_output"></a>
### Simulating with output file paths specified via `output_paths`

As expected, one may use both relative and absolute paths to configure where the output files of the Monte Carlo simulation is going to be created; in the example above, we choose relative paths to the directory that hosts the Jupyter notebook file where `battery_parallel_MC.simulate` is executed.

In [10]:
output_paths = ["A","B"]

battery_parallel_MC.simulate(models, simulation_configs, output_paths)

In [11]:
!head -n 12 A

H,H,H,H,T,H,T,H,H,H,T,H,T,T,H,H
T,H,H,H,H,H,H,T,T,T,H,H,H,T,T,H
T,H,T,T,T,H,H,T,T,H,H,T,H,H,H,T
T,H,T,H,T,T,T,T,H,H,H,H,H,T,T,T
T,H,T,H,H,H,H,T,H,H,H,T,H,T,H,T
H,T,T,T,T,T,H,H,T,H,H,H,T,H,H,T
T,T,T,T,T,T,H,H,T,T,H,T,H,H,H,H
H,H,T,T,H,H,H,T,H,H,T,H,H,T,H,H
H,H,H,H,T,H,H,T,H,H,T,H,H,H,H,H
T,T,T,H,T,H,T,H,T,T,T,H,T,H,H,T
H,H,H,T,H,H,T,H,T,H,T,T,T,T,T,T
T,H,H,H,T,H,H,H,T,T,T,H,T,T,T,T


In [12]:
!head -n 12 B

H,H,H,T,H,H,H,H,H,T,T,H,H,H,H,H,H,T,H,H,H,H,H,H,H,H,H,H,T,H,H,T
H,T,H,T,T,H,H,H,H,H,T,H,T,H,T,T,T,T,H,H,H,H,H,H,T,H,H,H,T,H,H,H
H,H,H,H,T,H,H,H,T,T,T,H,H,H,H,H,H,H,H,H,T,T,T,T,T,H,T,T,T,H,H,H
H,T,T,H,H,T,H,T,T,H,T,T,T,H,H,T,H,T,H,H,T,H,T,T,T,T,T,H,H,H,H,H
H,H,H,H,H,T,H,H,H,H,H,H,T,T,T,H,H,H,H,H,T,H,H,H,H,H,H,H,T,T,H,H
H,H,H,H,H,H,H,H,H,H,H,T,H,H,T,T,H,H,T,H,H,H,H,H,T,H,T,H,H,T,H,H
H,H,H,H,T,T,H,T,H,T,H,H,H,H,T,T,T,H,H,H,T,T,T,T,H,T,H,H,H,T,H,H
T,H,T,H,T,H,T,T,T,T,T,T,H,H,H,H,H,T,T,H,H,T,H,T,H,H,H,T,T,T,H,T
T,T,T,H,H,T,H,T,H,H,H,H,T,T,H,H,H,H,H,T,T,H,H,H,H,H,H,H,T,H,H,T
H,H,T,H,T,H,H,T,T,T,H,H,T,H,T,H,T,H,H,H,H,T,H,H,H,T,H,H,H,H,H,H
H,H,T,H,H,T,H,H,H,H,H,H,H,T,T,H,H,H,H,H,H,H,T,H,H,H,H,H,H,H,H,T
H,H,T,T,H,H,T,T,H,H,H,H,H,H,T,T,H,T,H,T,H,T,H,H,T,H,H,T,H,T,H,H


<a id="simulate_start"></a>
### Simulating by completing a pre-existing sequence with `starting_point`

To allow to complete a pre-existing sequence of heads 'H' and tails 'T', we first modify the `models`

In [13]:
def CoinSequence(number_flips, rng, bias, initial_sequence):
  sequence_completion = ["H" if rng.random() <= bias else "T" for _ in range(number_flips)]

  full_sequence = initial_sequence + sequence_completion

  return full_sequence

models = [CoinSequence, CoinSequence]

Then, we may include the initial sequence via the parameter `starting_point`:

In [14]:
# 200,000 sequences, each starting with 4 'T' and followed by 12 'H' or 'T' generated by simulating tossing a fair coin
fair_coin_config = {"parameters": 0.5, "number_simulations" : 200000, "number_points": 12, "starting_point": ['T', 'T', 'T', 'T', 'T']}

# 80,000 sequences, starting with 9 'T' for 'H' and followed by 28 'H' or 'T' generated by simulating tossing a biased coin
biased_coin_config = {"parameters": 0.7, "number_simulations" : 80000, "number_points": 28, "starting_point": ['T', 'T', 'T', 'T', 'T']}

simulation_configs = [fair_coin_config, biased_coin_config]

We pick the paths to the output files to be relative to the directory where the file containing `battery_parallel_MC.simulate` is executed, knowing that in case the directories/files do not exist, they will be **created**, and if they do exist, they will be **overwritten**.

In [15]:
output_paths = ["fair_coin/simulation_1.txt", "biased_coin/simulation_1.txt"]

battery_parallel_MC.simulate(models, simulation_configs, output_paths)

In [16]:
!head -n 12 fair_coin/simulation_1.txt

T,T,T,T,T,H,H,H,H,T,H,T,H,H,H,T,H
T,T,T,T,T,T,T,H,H,T,H,H,H,H,H,H,T
T,T,T,T,T,T,T,H,H,H,T,T,H,T,H,T,T
T,T,T,T,T,T,H,H,T,T,H,H,T,H,H,H,T
T,T,T,T,T,T,H,T,H,T,T,T,T,H,H,H,H
T,T,T,T,T,H,T,T,T,T,H,T,H,H,H,H,T
T,T,T,T,T,H,H,H,T,H,T,H,T,H,T,T,T
T,T,T,T,T,T,T,H,H,T,H,H,H,T,H,H,T
T,T,T,T,T,T,T,T,T,T,T,H,H,T,T,H,T
T,T,T,T,T,H,H,H,H,H,H,T,T,H,H,H,T
T,T,T,T,T,H,H,T,H,H,T,H,H,H,H,H,H
T,T,T,T,T,T,H,H,T,H,H,T,H,H,H,H,H


In [17]:
!head -n 12 biased_coin/simulation_1.txt

T,T,T,T,T,H,H,H,T,H,H,H,H,H,T,T,H,H,H,H,H,H,T,H,H,H,H,H,H,H,H,H,H
T,T,T,T,T,T,H,H,T,H,T,H,T,T,H,H,H,H,H,T,H,T,H,T,T,T,T,H,H,H,H,H,H
T,T,T,T,T,T,H,H,H,T,H,H,H,H,H,H,H,T,H,H,H,T,T,T,H,H,H,H,H,H,H,H,H
T,T,T,T,T,T,T,T,T,T,H,T,T,T,H,H,H,H,T,T,H,H,T,H,T,T,H,T,T,T,H,H,T
T,T,T,T,T,H,T,H,H,T,H,T,T,T,T,T,H,H,H,H,H,H,H,H,H,H,T,H,H,H,H,H,H
T,T,T,T,T,T,T,T,H,H,H,H,H,T,H,H,H,H,H,H,H,T,T,H,H,H,H,H,H,H,H,H,H
T,T,T,T,T,H,H,H,T,H,H,T,T,H,H,T,H,H,H,H,H,T,H,T,H,H,T,H,H,H,H,H,H
T,T,T,T,T,T,T,H,T,H,T,H,H,H,H,T,T,T,H,H,H,T,T,T,T,H,T,H,H,H,T,H,H
T,T,T,T,T,T,H,T,H,T,H,T,T,T,T,T,T,H,H,H,H,H,T,T,H,H,T,H,T,H,H,H,T
T,T,T,T,T,T,T,H,T,T,T,T,H,H,T,H,T,H,H,H,H,T,T,H,H,H,H,H,T,T,H,H,H
T,T,T,T,T,H,H,H,H,T,H,H,T,H,H,T,H,T,H,H,T,T,T,H,H,T,H,T,H,T,H,H,H
T,T,T,T,T,H,T,H,H,H,T,H,H,H,H,H,H,H,H,T,H,H,T,H,H,H,H,H,H,H,T,T,H


However, note that in this example the generation of the subsequent points in a simulation is independent of the **initial sequence** enclosed in the `starting_point`. In the next notebook, we will implement an autoregressive model, which might use e.g. the present price of a stock as `starting_point`.